# Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import scipy
import statsmodels as sm

from pyjstat import pyjstat
from ecbdata import ecbdata
import requests

from bs4 import BeautifulSoup
import re

from obiwan import *

import os
import warnings
warnings.filterwarnings("ignore")

# Load data

# Enquadramento do contexto

In [2]:
series_id_dict = {
    12587303: 'Custo dos financiamentos PME',
    12587294: 'Rendibilidade do Ativo PME',
    12458144: 'Endividamento das pequenas empresas não financeiras privadas',
    12587298: 'Autonomia financeira PME',
    12587273: 'Financiamentos obtidos PME'
    }

In [3]:
series_id_dict.keys()

dict_keys([12587303, 12587294, 12458144, 12587298, 12587273])

In [4]:
series_id_dict.values()

dict_values(['Custo dos financiamentos PME', 'Rendibilidade do Ativo PME', 'Endividamento das pequenas empresas não financeiras privadas', 'Autonomia financeira PME', 'Financiamentos obtidos PME'])

In [6]:
series_id = 12587303
#variable_name = None
variable_name = series_id_dict[series_id]
extract_data_from_bank_pt(series_id, variable_name).head(3)

,Date,Custo dos financiamentos PME
0,2006-12-31,4.6
1,2007-03-31,4.7
2,2007-06-30,4.9


In [7]:
dfs_dict = {}

for series_id, variable_name in series_id_dict.items():
    dfs_dict[variable_name] = extract_data_from_bank_pt(series_id, variable_name)

In [8]:
dfs_dict.keys()

dict_keys(['Custo dos financiamentos PME', 'Rendibilidade do Ativo PME', 'Endividamento das pequenas empresas não financeiras privadas', 'Autonomia financeira PME', 'Financiamentos obtidos PME'])

In [9]:
dfs_dict['Rendibilidade do Ativo PME']

,Date,Rendibilidade do Ativo PME
0,2006-12-31,6.7
1,2007-03-31,7.4
2,2007-06-30,7.5
3,2007-09-30,8.0
4,2007-12-31,8.2
...,...,...
65,2023-03-31,8.1
66,2023-06-30,8.1
67,2023-09-30,8.0
68,2023-12-31,7.7


Extract manually the xls from the following URL:

https://www.eba.europa.eu/risk-and-data-analysis/risk-analysis/risk-monitoring/risk-dashboard

Excel auxiliar ao gráfico de parâmetros de risco da eba é realizado em:

	Análise KRI - Risk parameters annex - Resumo que consta do seguinte caminho:Novo Banco S.A\GVM Team - General\08_benchmarking\EBA\Risk Parameters

In [10]:
KRI_analysis_risk_parameters_annex(2020)

File downloaded successfully as KRI - Risk parameters annex - Q4 2020.xlsx.


If it'didnt downloaded the right file, please download it manually for the right specific year and quarter on the url below:
      https://www.eba.europa.eu/risk-and-data-analysis/risk-analysis/risk-monitoring/risk-dashboard


# Producing outputs

In [11]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [12]:
# df_base.columns

# sheet LongRun_Data_Total (P)

In [13]:
pd_list = [19.98, 0.40, 0.77, 1.23, 1.90, 2.94, 4.25, 7.68, 28.96, 12.08]

In [14]:
drs = pd.read_csv('drs.csv')
drs['data'] = drs['data'].astype(int)

bons = pd.read_csv('bons.csv')
bons['data'] = bons['data'].astype(int)

defaults = pd.read_csv('defaults.csv')
defaults['data'] = defaults['data'].astype(int)

In [15]:
def get_first_four_digits(x):
    return int(str(x)[:4])

In [16]:
df_bons_aux = bons
df_bons_aux['year'] = df_bons_aux['data'].apply(get_first_four_digits)

bons_yearly_avg_cardinalities = df_bons_aux.drop(columns=['data']).groupby('year').mean()#.round(1)
bons_yearly_avg_cardinalities.head(2)

,<=500_1,>500_Rated_1,>500_Rated_2,>500_Rated_3,>500_Rated_4,>500_Rated_5,>500_Rated_6,>500_Rated_7,>500_Rated_8,>500_Unrated_1,Rated >500,TOTAL
year,,,,,,,,,,,,
2010,1423.083333,2019.333333,1733.833333,2205.5,2340.083333,2284.583333,2898.250000,2449.416667,2115.583333,3570.916667,18046.583333,23040.583333
2011,1589.416667,2323.416667,1875.333333,2344.0,2355.833333,2217.500000,2676.833333,2151.583333,1995.750000,3278.000000,17940.250000,22807.666667


In [17]:
df_defaults_aux = defaults
df_defaults_aux['year'] = df_defaults_aux['data'].apply(get_first_four_digits)

defaults_yearly_avg_cardinalities = df_defaults_aux.drop(columns=['data']).groupby('year').mean()#.round(1).astype(int)
defaults_yearly_avg_cardinalities.head(2)

,<=500_1,>500_Rated_1,>500_Rated_2,>500_Rated_3,>500_Rated_4,>500_Rated_5,>500_Rated_6,>500_Rated_7,>500_Rated_8,>500_Unrated_1,Rated >500,TOTAL
year,,,,,,,,,,,,
2010,104.25,5.666667,13.083333,27.166667,42.583333,62.833333,122.75,160.916667,530.333333,254.25,965.333333,1323.833333
2011,161.75,10.583333,19.666667,49.083333,75.583333,100.833333,166.25,208.583333,643.666667,268.75,1274.250000,1704.750000


In [18]:
drs_avg_yearly = (defaults_yearly_avg_cardinalities / bons_yearly_avg_cardinalities).fillna(0)*100
drs_avg_yearly.head()

,<=500_1,>500_Rated_1,>500_Rated_2,>500_Rated_3,>500_Rated_4,>500_Rated_5,>500_Rated_6,>500_Rated_7,>500_Rated_8,>500_Unrated_1,Rated >500,TOTAL
year,,,,,,,,,,,,
2010,7.325643,0.280621,0.754590,1.231769,1.819736,2.750319,4.235314,6.569591,25.067948,7.120021,5.349120,5.745659
2011,10.176690,0.455507,1.048702,2.093999,3.208348,4.547163,6.210697,9.694411,32.251869,8.198597,7.102744,7.474460
2012,9.886259,0.374545,1.196183,2.059545,2.651086,3.432936,5.401140,8.412949,28.803132,8.480824,6.354363,7.044373
2013,8.006587,0.318672,0.724367,1.007517,1.559493,2.830229,3.371807,5.315486,20.798927,7.668397,4.407420,5.264727
2014,7.691293,0.361470,0.631377,0.730316,1.044646,1.688428,2.699233,5.566704,20.485845,8.635097,4.015700,5.028894


In [19]:
bons_yearly_avg_cardinalities = bons_yearly_avg_cardinalities.reset_index()
drs_avg_yearly = drs_avg_yearly.reset_index()
defaults_yearly_avg_cardinalities = defaults_yearly_avg_cardinalities.reset_index()

In [20]:
df_summary = pd.DataFrame({'Average Obs': bons_yearly_avg_cardinalities[bons_yearly_avg_cardinalities['year'] > 2010].mean()})

In [21]:
df_summary['Average Dfts'] = defaults_yearly_avg_cardinalities[defaults_yearly_avg_cardinalities['year'] > 2010].mean()

In [22]:
df_summary['Average DR'] = drs_avg_yearly[drs_avg_yearly['year'] > 2010].mean()

In [23]:
df_summary

,Average Obs,Average Dfts,Average DR
year,2016.500000,2016.500000,2016.500000
<=500_1,2281.691288,238.744318,10.871002
>500_Rated_1,2291.834596,4.474242,0.197353
>500_Rated_2,1779.847077,8.223970,0.457180
>500_Rated_3,2214.363102,17.515346,0.784279
>500_Rated_4,2228.677448,24.909188,1.117255
>500_Rated_5,2049.760975,37.078040,1.798664
>500_Rated_6,2401.881410,62.506265,2.529695
>500_Rated_7,1783.989850,89.217754,4.722678
>500_Rated_8,1207.534916,261.649136,19.577531
